In [1]:
# Load packages and data
import pandas as pd
import geopandas as gpd
import numpy as np
import os

In [2]:
prototypes_df  = pd.read_excel(r"W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\התפלגות גילים\backround_files\age_des_types.xlsx")
file_path=r"W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\forecast_2020_230720.xlsx"
data_df = pd.read_excel(file_path)

In [3]:
filename = os.path.basename(file_path)

# Remove the file extension if present
filename = os.path.splitext(filename)[0]

In [4]:
sectors=gpd.read_file(r'W:\Model Versions\4.0\TAZ\TAZ_V4_230518_Published.shp')

In [5]:
data_df=sectors.merge(data_df,left_on='Taz_num',right_on='TAZ',how='right')

In [6]:
data_df['sector_for_age']=data_df['sector']

In [7]:
data_df.loc[
    (data_df['zonetype'] == 'Judea and Samaria') &
    (data_df['sector'] != 'U_Orthodox'),'sector_for_age'
]='hitnachlut'


In [8]:
lst_muni=['מעלה אדומים',
 'גבעת זאב',
 'הר אדר',
 '0',
 'מעלה אפרים',
 'מגילות ים המלח\r\n',
 'קרני שומרון',
 'אלפי מנשה',
 'בית אריה-עופרים',
 'ערבות הירדן\r\n',
 'אריאל']

In [9]:
data_df.loc[data_df['Muni_Heb'].isin(lst_muni),'sector_for_age'
]='Jewish'


In [10]:
data_df.loc[
    (data_df['sector'] == 'arabs_behined_seperation_wall') |
    (data_df['sector'] == 'Arab'),'sector_for_age'
]='Arab'


In [11]:
data_df=data_df.loc[
    (data_df['sector'] != 'Palestinian')
]


In [12]:
col=[ 'age0_4',
 'age5_9',
 'age10_14',
 'age15_19',
 'age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',
 'age45_49',
 'age50_54',
 'age55_59',
 'age60_64',
 'age65_69',
 'age70_74',
 'age75up']

In [13]:
# Convert counts to percentages
data_df_pre = data_df[col].apply(lambda x: x / x.sum(), axis=1)

In [14]:
data_df=data_df.join(data_df_pre,lsuffix='', rsuffix='_prec')

In [15]:
col_name_prototype=['pop_0','pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [16]:
col_name_row=['age0_4_prec',
 'age5_9_prec',
 'age10_14_prec',
 'age15_19_prec',
 'age20_24_prec',
 'age25_29_prec',
 'age30_34_prec',
 'age35_39_prec',
 'age40_44_prec',
 'age45_49_prec',
 'age50_54_prec',
 'age55_59_prec',
 'age60_64_prec',
 'age65_69_prec',
 'age70_74_prec',
 'age75up_prec']

In [17]:
ls_sector=['U_Orthodox', 'Jewish', 'hitnachlut', 'Arab']

In [18]:
for s in ls_sector:
    locals()['data_df_{}'.format(s)]=data_df.loc[data_df['sector_for_age']=='{}'.format(s)]
    locals()['prototypes_df_{}'.format(s)]=prototypes_df.loc[prototypes_df['sector']=='{}'.format(s)]

In [19]:
for s in ls_sector:
    for index, row in locals()['data_df_{}'.format(s)].iterrows():
        # Step 3: Calculate similarity scores
        similarity_scores = []
        for _, prototype_row in locals()['prototypes_df_{}'.format(s)].iterrows():
            prototype_age_distributions = prototype_row[col_name_prototype].values 
            data_age_distributions = row[col_name_row].values         
            similarity_score = np.linalg.norm(prototype_age_distributions - data_age_distributions)
            similarity_scores.append(similarity_score)

        # Step 4: Determine closest prototype
        closest_index = np.argmin(similarity_scores)
        closest_prototype = locals()['prototypes_df_{}'.format(s)].iloc[closest_index]['classification_name']
        

        # Assign closest_prototype to the corresponding row in data_df or perform any desired action

        locals()['data_df_{}'.format(s)].at[index,'classification_name'] = closest_prototype
        

C:\Users\gidon\Anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\gidon\AppData\Local\Temp\ipykernel_19212\2666082859.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locals()['data_df_{}'.format(s)].at[index,'classification_name'] = closest_prototype
C:\Users\gidon\Anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a c

In [20]:
data_df=pd.concat([data_df_U_Orthodox,data_df_Jewish,data_df_hitnachlut,data_df_Arab],axis=0)

In [21]:
data_df[['Taz_num','sector_for_age','classification_name']].to_excel(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\התפלגות גילים\backround_files\{}_age_dis_type.xlsx'.format(filename),index=False)